In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
parksfile = 'Data/parks.csv'
parks_df = pd.read_csv(parksfile)
parks_df.head()

,Park Code,Park Name,State,Acres,Latitude,Longitude
0,ACAD,Acadia National Park,ME,47390,44.35,-68.21
1,ARCH,Arches National Park,UT,76519,38.68,-109.57
2,BADL,Badlands National Park,SD,242756,43.75,-102.50
3,BIBE,Big Bend National Park,TX,801163,29.25,-103.25
4,BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [3]:
speciesfile = 'Data/species.csv'
species_df = pd.read_csv(speciesfile)
species_df.head()

C:\Users\Ryan\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Species ID,Park Name,Category,Order,Family,Scientific Name,Common Names,Record Status,Occurrence,Nativeness,Abundance,Seasonality,Conservation Status,Unnamed: 13
0,ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Approved,Present,Native,Rare,Resident,NaN,NaN
1,ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Approved,Present,Native,Abundant,NaN,NaN,NaN
2,ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Approved,Present,Not Native,Common,NaN,Species of Concern,NaN
3,ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Approved,Not Confirmed,Native,NaN,NaN,Endangered,NaN
4,ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Approved,Present,Unknown,Common,Breeder,NaN,NaN


In [4]:
parks_transformed = parks_df[['Park Code', 'Park Name', 'State', 'Acres', 'Latitude', 'Longitude']].copy()
parks_transformed = parks_transformed.rename(columns={'Park Code': 'park_code', 'Park Name': 'park_name', 'State': 'state', 'Acres': 'acres', 'Latitude': 'latitude', 'Longitude': 'longitude'})
parks_transformed.set_index('park_code', inplace=True)
parks_transformed.head()

,park_name,state,acres,latitude,longitude
park_code,,,,,
ACAD,Acadia National Park,ME,47390,44.35,-68.21
ARCH,Arches National Park,UT,76519,38.68,-109.57
BADL,Badlands National Park,SD,242756,43.75,-102.50
BIBE,Big Bend National Park,TX,801163,29.25,-103.25
BISC,Biscayne National Park,FL,172924,25.65,-80.08


In [5]:
species_transformed = species_df[['Species ID', 'Park Name', 'Category', 'Order', 'Family', 'Scientific Name', 'Common Names', 'Occurrence', 'Nativeness']].copy()
species_transformed = species_transformed.rename(columns={'Species ID': 'species_id', 'Park Name': 'park_name', 'Category': 'category', 'Order': 'order', 'Family': 'family', 'Scientific Name': 'scientific_name', 'Common Names': 'common_names', 'Occurrence': 'occurrence', 'Nativeness': 'nativeness'})
species_transformed.set_index('species_id', inplace=True)
species_transformed.head()

,park_name,category,order,family,scientific_name,common_names,occurrence,nativeness
species_id,,,,,,,,
ACAD-1000,Acadia National Park,Mammal,Artiodactyla,Cervidae,Alces alces,Moose,Present,Native
ACAD-1001,Acadia National Park,Mammal,Artiodactyla,Cervidae,Odocoileus virginianus,"Northern White-Tailed Deer, Virginia Deer, Whi...",Present,Native
ACAD-1002,Acadia National Park,Mammal,Carnivora,Canidae,Canis latrans,"Coyote, Eastern Coyote",Present,Not Native
ACAD-1003,Acadia National Park,Mammal,Carnivora,Canidae,Canis lupus,"Eastern Timber Wolf, Gray Wolf, Timber Wolf",Not Confirmed,Native
ACAD-1004,Acadia National Park,Mammal,Carnivora,Canidae,Vulpes vulpes,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Present,Unknown


In [6]:
connection_string = "postgres:admin@localhost:5432/species_db"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
engine.table_names()

['park_species', 'parks']

In [8]:
species_transformed.to_sql(name='park_species', con=engine, if_exists='replace', index=True)

InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table park_species because other objects depend on it
DETAIL:  view speciesandparks depends on table park_species
view parkspecies depends on table park_species
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE park_species]
(Background on this error at: http://sqlalche.me/e/13/2j85)

In [9]:
parks_transformed.to_sql(name='parks', con=engine, if_exists='replace', index=True)

InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table parks because other objects depend on it
DETAIL:  view speciesandparks depends on table parks
view parkspecies depends on table parks
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE parks]
(Background on this error at: http://sqlalche.me/e/13/2j85)

In [10]:
parks_transformed.index

Index(['ACAD', 'ARCH', 'BADL', 'BIBE', 'BISC', 'BLCA', 'BRCA', 'CANY', 'CARE',
       'CAVE', 'CHIS', 'CONG', 'CRLA', 'CUVA', 'DENA', 'DEVA', 'DRTO', 'EVER',
       'GAAR', 'GLAC', 'GLBA', 'GRBA', 'GRCA', 'GRSA', 'GRSM', 'GRTE', 'GUMO',
       'HALE', 'HAVO', 'HOSP', 'ISRO', 'JOTR', 'KATM', 'KEFJ', 'KOVA', 'LACL',
       'LAVO', 'MACA', 'MEVE', 'MORA', 'NOCA', 'OLYM', 'PEFO', 'PINN', 'REDW',
       'ROMO', 'SAGU', 'SEKI', 'SHEN', 'THRO', 'VOYA', 'WICA', 'WRST', 'YELL',
       'YOSE', 'ZION'],
      dtype='object', name='park_code')

In [ ]:
natpark_key='WWcgmfe0hephwhVSITQXQHCy5bfr0pBQSFgOoHog'


In [2]:
https://developer.nps.gov/api/v1/alerts?parkCode=acad,dena&api_key=WWcgmfe0hephwhVSITQXQHCy5bfr0pBQSFgOoHog

SyntaxError: invalid syntax (<ipython-input-2-334ef69a9654>, line 1)

In [12]:
import requests
import json

In [24]:
stuff= requests.get('https://developer.nps.gov/api/v1/alerts?parkCode=ACAD,dena&api_key=WWcgmfe0hephwhVSITQXQHCy5bfr0pBQSFgOoHog')

In [25]:
stuff_json=stuff.json()

In [26]:
print(json.dumps(stuff_json, indent=4, sort_keys=True))

{
    "data": [
        {
            "category": "Information",
            "description": "The Denali Park Road is currently open to Mile 3, Park Headquarters. Wintry conditions beyond that point prevent vehicle travel, though pedestrian travel is permitted.",
            "id": "367A20CE-5A30-4E35-B586-09BC23472E83",
            "lastIndexedDate": "2020-12-28 18:16:18.0",
            "parkCode": "dena",
            "title": "Road Open To: Mile 3 (Park Headquarters)",
            "type": "Information",
            "url": ""
        },
        {
            "category": "Park Closure",
            "description": "Facilities and services in Denali remain altered due to the pandemic. Find out which facilities are closed, which are open but with different hours than normal, and which are unaffected.",
            "id": "144239D9-06BD-4FA4-BE9E-85F4B028A373",
            "lastIndexedDate": "2020-09-21 12:26:11.0",
            "parkCode": "dena",
            "title": "Changed Hours & Some Cl